In [1]:
import anndata
import scvelo as scv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc

In [2]:
results_file = '/storage/holab/linxy/vivian/scanpy/ncc_seurate.h5ad'  # the file that will store the analysis results

In [3]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.8.2 anndata==0.7.8 umap==0.5.3 numpy==1.20.3 scipy==1.6.2 pandas==1.2.4 scikit-learn==0.24.1 statsmodels==0.12.2 pynndescent==0.5.6


In [4]:
# get meta-data information
path = "/storage/holab/linxy/vivian/veloAE/"
cellID_obs = pd.read_csv(path + "ncc_cellID_obs.csv")
print(cellID_obs.head())
print(len(cellID_obs))
umap_cord = pd.read_csv(path + "ncc_cell_embeddings.csv")
print(umap_cord)
cell_clusters = pd.read_csv(path + "ncc_clusters.csv")

                         x
0  ncc1_AAACCCATCAGCAATC-1
1  ncc1_AAACGAACACGGTGCT-1
2  ncc1_AAACGAACATTGTCGA-1
3  ncc1_AAACGCTCAAGAGGTC-1
4  ncc1_AAACGCTGTGCCCGTA-1
7357
                   Unnamed: 0    UMAP_1    UMAP_2
0     ncc1_AAACCCATCAGCAATC-1  0.913846  0.550288
1     ncc1_AAACGAACACGGTGCT-1 -1.886460  2.125017
2     ncc1_AAACGAACATTGTCGA-1  4.905686  0.409525
3     ncc1_AAACGCTCAAGAGGTC-1  3.466465  0.183858
4     ncc1_AAACGCTGTGCCCGTA-1 -4.015033 -5.032781
...                       ...       ...       ...
7352  ncc4_TTTGGAGTCAAGTCGT-1  3.743522  0.089498
7353  ncc4_TTTGGTTGTTTGCCGG-1 -5.931196  0.739618
7354  ncc4_TTTGGTTTCTCCTACG-1 -3.608194 -5.238413
7355  ncc4_TTTGTTGCACACGTGC-1 -0.909402 -0.013683
7356  ncc4_TTTGTTGGTTTCCAAG-1 -4.805133 -1.220025

[7357 rows x 3 columns]


In [5]:
sampleL = ["ncc1", "ncc2", "ncc3", "ncc4"]
adata = sc.read_10x_mtx('/storage/holab/linxy/vivian/' + sampleL[0] + '_cellranger/outs/filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)                             
ldata =  scv.read("/storage/holab/linxy/vivian/" + sampleL[0] + "_cellranger/velocyto/" + sampleL[0] + "_cellranger.loom", cache=True)
adata.var_names_make_unique()
adata = scv.utils.merge(adata, ldata)
adata.obs_names = sampleL[0] + '_' + adata.obs_names
print(adata)
print(adata.obs_names)

... reading from cache file cache/storage-holab-linxy-vivian-ncc1_cellranger-outs-filtered_feature_bc_matrix-matrix.h5ad
... reading from cache file cache/storage-holab-linxy-vivian-ncc1_cellranger-velocyto-ncc1_cellranger.h5ad


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 2506 × 36601
    obs: 'Clusters', '_X', '_Y', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size'
    var: 'gene_ids', 'feature_types', 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'
Index(['ncc1_AAACCCATCAGCAATC', 'ncc1_AAACGAACACGGTGCT',
       'ncc1_AAACGAACATTGTCGA', 'ncc1_AAACGCTCAAGAGGTC',
       'ncc1_AAACGCTGTGCCCGTA', 'ncc1_AAAGAACCAGAGGTAC',
       'ncc1_AAAGAACTCTTGCAAG', 'ncc1_AAAGGATCACCATAAC',
       'ncc1_AAAGGATCATGGCGCT', 'ncc1_AAAGGGCTCTTGGCTC',
       ...
       'ncc1_TTTGACTTCAAGTGTC', 'ncc1_TTTGATCAGAGCGACT',
       'ncc1_TTTGGAGCATACAGAA', 'ncc1_TTTGGAGTCAAGTCGT',
       'ncc1_TTTGGTTGTTTGCCGG', 'ncc1_TTTGGTTTCTCCTACG',
       'ncc1_TTTGTTGAGCATGATA', 'ncc1_TTTGTTGAGGTTGTTC',
       'ncc1_TTTGTTGCACACGTGC', 'ncc1_TTTGTTGGTTTCCAAG'],
      dtype='object', length=2506)


In [6]:
for i in range(1,len(sampleL)):
    tempadata = sc.read_10x_mtx('/storage/holab/linxy/vivian/' + sampleL[i] + '_cellranger/outs/filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)                             
    templdata =  scv.read("/storage/holab/linxy/vivian/" + sampleL[i] + "_cellranger/velocyto/" + sampleL[i] + "_cellranger.loom", cache=True)
    tempadata.var_names_make_unique()
    tempadata = scv.utils.merge(tempadata, templdata)
    tempadata.obs_names = sampleL[i] + '_' + tempadata.obs_names
    adata = adata.concatenate(tempadata, join='outer', index_unique=None)

print(ldata)

... reading from cache file cache/storage-holab-linxy-vivian-ncc2_cellranger-outs-filtered_feature_bc_matrix-matrix.h5ad
... reading from cache file cache/storage-holab-linxy-vivian-ncc2_cellranger-velocyto-ncc2_cellranger.h5ad


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


... reading from cache file cache/storage-holab-linxy-vivian-ncc3_cellranger-outs-filtered_feature_bc_matrix-matrix.h5ad
... reading from cache file cache/storage-holab-linxy-vivian-ncc3_cellranger-velocyto-ncc3_cellranger.h5ad


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


... reading from cache file cache/storage-holab-linxy-vivian-ncc4_cellranger-outs-filtered_feature_bc_matrix-matrix.h5ad
... reading from cache file cache/storage-holab-linxy-vivian-ncc4_cellranger-velocyto-ncc4_cellranger.h5ad


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 2506 × 36601
    obs: 'Clusters', '_X', '_Y', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


## barcodes

In [8]:
## change obs_names
print(adata.obs_names)
print(np.unique(adata.obs_names).size == len(adata.obs_names))

Index(['ncc1_AAACCCATCAGCAATC', 'ncc1_AAACGAACACGGTGCT',
       'ncc1_AAACGAACATTGTCGA', 'ncc1_AAACGCTCAAGAGGTC',
       'ncc1_AAACGCTGTGCCCGTA', 'ncc1_AAAGAACCAGAGGTAC',
       'ncc1_AAAGAACTCTTGCAAG', 'ncc1_AAAGGATCACCATAAC',
       'ncc1_AAAGGATCATGGCGCT', 'ncc1_AAAGGGCTCTTGGCTC',
       ...
       'ncc4_TTTGACTTCAAGTGTC', 'ncc4_TTTGATCAGAGCGACT',
       'ncc4_TTTGGAGCATACAGAA', 'ncc4_TTTGGAGTCAAGTCGT',
       'ncc4_TTTGGTTGTTTGCCGG', 'ncc4_TTTGGTTTCTCCTACG',
       'ncc4_TTTGTTGAGCATGATA', 'ncc4_TTTGTTGAGGTTGTTC',
       'ncc4_TTTGTTGCACACGTGC', 'ncc4_TTTGTTGGTTTCCAAG'],
      dtype='object', length=9892)
True


In [11]:
## change cellID name in metadata
cellID_obs['x'] = cellID_obs['x'].str.replace('-1', '')
print(np.unique(cellID_obs['x']).size == len(cellID_obs['x']))  ## check whether have duplicate barcodes
print(len(cellID_obs))
print(cellID_obs)

True
7357
                          x
0     ncc1_AAACCCATCAGCAATC
1     ncc1_AAACGAACACGGTGCT
2     ncc1_AAACGAACATTGTCGA
3     ncc1_AAACGCTCAAGAGGTC
4     ncc1_AAACGCTGTGCCCGTA
...                     ...
7352  ncc4_TTTGGAGTCAAGTCGT
7353  ncc4_TTTGGTTGTTTGCCGG
7354  ncc4_TTTGGTTTCTCCTACG
7355  ncc4_TTTGTTGCACACGTGC
7356  ncc4_TTTGTTGGTTTCCAAG

[7357 rows x 1 columns]


In [13]:
filtered_barcode = np.intersect1d(cellID_obs['x'],adata.obs_names)
len(filtered_barcode)

7357

In [14]:
filtered_adata = adata[filtered_barcode].copy()
print(filtered_adata)

AnnData object with n_obs × n_vars = 7357 × 36601
    obs: 'Clusters', '_X', '_Y', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'batch'
    var: 'gene_ids', 'feature_types', 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


Add UMAP and annotation

In [18]:
adata_index = pd.DataFrame(filtered_adata.obs.index)
adata_index.columns = ['CellID']
print(adata_index)

                     CellID
0     ncc1_AAACCCATCAGCAATC
1     ncc1_AAACGAACACGGTGCT
2     ncc1_AAACGAACATTGTCGA
3     ncc1_AAACGCTCAAGAGGTC
4     ncc1_AAACGCTGTGCCCGTA
...                     ...
7352  ncc4_TTTGGAGTCAAGTCGT
7353  ncc4_TTTGGTTGTTTGCCGG
7354  ncc4_TTTGGTTTCTCCTACG
7355  ncc4_TTTGTTGCACACGTGC
7356  ncc4_TTTGTTGGTTTCCAAG

[7357 rows x 1 columns]


In [19]:
umap_cord = umap_cord.rename(columns = {'Unnamed: 0':'CellID'})
#umap_cord = umap_cord.rename(columns = {'Cell ID':'CellID'})
umap_cord['CellID'] = umap_cord['CellID'].str.replace('-1', '')
umap_ordered = adata_index.merge(umap_cord, on = "CellID")
print(umap_cord)

                     CellID    UMAP_1    UMAP_2
0     ncc1_AAACCCATCAGCAATC  0.913846  0.550288
1     ncc1_AAACGAACACGGTGCT -1.886460  2.125017
2     ncc1_AAACGAACATTGTCGA  4.905686  0.409525
3     ncc1_AAACGCTCAAGAGGTC  3.466465  0.183858
4     ncc1_AAACGCTGTGCCCGTA -4.015033 -5.032781
...                     ...       ...       ...
7352  ncc4_TTTGGAGTCAAGTCGT  3.743522  0.089498
7353  ncc4_TTTGGTTGTTTGCCGG -5.931196  0.739618
7354  ncc4_TTTGGTTTCTCCTACG -3.608194 -5.238413
7355  ncc4_TTTGTTGCACACGTGC -0.909402 -0.013683
7356  ncc4_TTTGTTGGTTTCCAAG -4.805133 -1.220025

[7357 rows x 3 columns]


In [21]:
umap_ordered = umap_ordered.iloc[:,1:]
filtered_adata.obsm['X_umap'] = umap_ordered.values

In [23]:
cell_clusters = cell_clusters.rename(columns = {'Unnamed: 0':'CellID'})
cell_clusters['CellID'] = cell_clusters['CellID'].str.replace('-1', '')
cell_clusters = adata_index.merge(cell_clusters, on = "CellID")
print(cell_clusters)

                     CellID  x
0     ncc1_AAACCCATCAGCAATC  7
1     ncc1_AAACGAACACGGTGCT  1
2     ncc1_AAACGAACATTGTCGA  0
3     ncc1_AAACGCTCAAGAGGTC  0
4     ncc1_AAACGCTGTGCCCGTA  7
...                     ... ..
7352  ncc4_TTTGGAGTCAAGTCGT  0
7353  ncc4_TTTGGTTGTTTGCCGG  5
7354  ncc4_TTTGGTTTCTCCTACG  7
7355  ncc4_TTTGTTGCACACGTGC  1
7356  ncc4_TTTGTTGGTTTCCAAG  4

[7357 rows x 2 columns]


In [24]:
cell_clusters = cell_clusters.iloc[:,1:]
filtered_adata.obs['seurat_clusters'] = cell_clusters.values
print(filtered_adata)

AnnData object with n_obs × n_vars = 7357 × 36601
    obs: 'Clusters', '_X', '_Y', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'batch', 'seurat_clusters'
    var: 'gene_ids', 'feature_types', 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


In [26]:
filtered_adata.write(results_file)

... storing 'feature_types' as categorical
